In [ ]:
import urllib2
from bs4 import BeautifulSoup
import urllib
from PIL import Image
import matplotlib.pyplot as plt
import logging
import time

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s %(message)s',
                    datefmt='%m-%d %H:%M')

request_root_url = "http://www.sybj.com"
request_headers = {'User-Agent' : 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'}
sleep_second = 3

def get_html(page_url):
    url = request_root_url + page_url
    request = urllib2.Request(url, headers=request_headers)
    response = urllib2.urlopen(request, timeout=60)
    html = response.read()
    return html

def save_html(html, filename):
    fhtml = open(filename, 'w')
    fhtml.write(html)
    fhtml.close()
    
def get_img_url(soup):
    img_tag = soup.find("img", id="imgSybj")
    if img_tag is None:
        return None
    else:
        return img_tag.get("src")
    
def get_zan_num(soup):
    zan_num_tag = soup.find("span", id="zan-num")
    if zan_num_tag is None:
        return None
    else:
        return zan_num_tag.string
    
def get_cai_num(soup):
    cai_num_tag = soup.find("span", id="cai-num")
    if cai_num_tag is None:
        return None
    else:
        return cai_num_tag.string
    
def get_prev_page(soup):
    prev_page_tag = soup.find("a", id="prepage")
    if prev_page_tag is None:
        return None
    else:
        return prev_page_tag.get("href")
    
def save_image(src_url, trg_name):
    urllib.urlretrieve(src_url, trg_name)
    
def get_img_id(url):
    _idx = url.find("&id=") + 4
    img_id = url[_idx:]
    return img_id
        

In [ ]:
if __name__ == "__main__":
    first_page = "/may.php?c=w&a=organizationCommunity&t=1&hid=1126&id=405"
    href = first_page
    #while href is not None:
    for _ in range(10):
        img_id = get_img_id(href)
        if (not href.startswith("/may.php?")) or (img_id == ""):
            break
        
        try:
            logging.info("process url=%s%s"%(request_root_url, href))
            
            html = get_html(href)
            soup = BeautifulSoup(html, 'html.parser')
            save_html(html, "./data/cache/%s.html"%img_id)
            logging.info("[html]./data/cache/%s.html has been saved."%img_id)
                
            img_url = get_img_url(soup)
            zan_num = get_zan_num(soup)
            cai_num = get_cai_num(soup)
            img_filename = "./data/img/%s_%s_%s.jpg" % (img_id, zan_num, cai_num)
            save_image(img_url, img_filename)
            logging.info("[image]%s has been saved."%img_filename)
        except Exception, e:
            logging.error(e.message)
            time.sleep(sleep_second*10)
            continue
        
        href = get_prev_page(soup)
        time.sleep(sleep_second)

In [1]:
from bs4 import BeautifulSoup
from multiprocessing import Process, Semaphore, Lock, Queue, Pool
import os

path = "./data/cache/"

img_tags_file = open("./data/img_tags.csv", 'w')
img_author_file = open("./data/img_author.csv", 'w')

img_tags_lock = Lock()
img_author_lock = Lock()

def Parser(html_file):
    imgid = os.path.splitext(html_file)[0]
    html_file = path + html_file
    html = open(html_file, 'r').read()
    soup = BeautifulSoup(html, "html.parser")
    
    author_div = soup.find("div", class_="auther")
    tag_div = soup.find("div", text=u"标签")
    
    if tag_div is not None:
        try:
            img_tags_lock.acquire()
            tag = tag_div.parent.a.div.string.strip().encode("gbk")
            img_tags_file.write("%s\t%s\n"%(imgid, tag))
            img_tags_file.flush()
        except:
            pass
        finally:
            img_tags_lock.release()
    
    if author_div is not None:
        try:
            img_author_lock.acquire()
            author = author_div.a.string.strip().encode("gbk")
            img_author_file.write("%s\t%s\n"%(imgid, author))
            img_author_file.flush()
        except:
            pass
        finally:
            img_author_lock.release()
    
p = Pool(processes=16)
p.map(Parser, os.listdir(path))
p.close()
p.join()
    
img_tags_file.flush()
img_author_file.flush()
img_tags_file.close()
img_author_file.close()

In [ ]:
print soup.find("div", text=u"标签").parent.a.div.string


In [ ]:
print soup.find("div", class_="auther").a.string.strip()